# Rozliczanie podatków od kryptowalut w Polsce

## Konfiguracja

In [11]:
ROK = 2024

## Importy

In [12]:
from kryptorozliczator.exchange_interfaces.zonda_interface import ZondaInterface

import pandas as pd

interfaces = [ZondaInterface()]

## Pobranie historii transakcji

In [ ]:
all_transactions_df = pd.DataFrame()

for interface in interfaces:
    transactions = interface.get_transaction_history(ROK)
    transactions_df = pd.DataFrame(transactions)
    transactions_df["exchange"] = interface.exchange_name
    all_transactions_df = pd.concat([all_transactions_df, transactions_df])



Fetching trades since 2024-01-01T00:00:00.000Z...
Reached end of relevant period or fetched less than limit.
Fetched 10 total trades for the year 2024.


,id,order,timestamp,datetime,symbol,type,side,price,amount,cost,takerOrMaker,fee,info,fees,exchange
0,bb916ef1-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.18,6.441520,129.989876,taker,"{'currency': 'RENDER', 'cost': 0.01288305}","{'id': 'bb916ef1-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.01288305}]",zonda
1,bb916ef2-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.19,43.960000,887.552400,taker,"{'currency': 'RENDER', 'cost': 0.08792}","{'id': 'bb916ef2-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.08792}]",zonda
2,bb916ef3-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.20,119.000000,2403.800000,taker,"{'currency': 'RENDER', 'cost': 0.238}","{'id': 'bb916ef3-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.238}]",zonda
3,bb919604-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.10,1.523223,32.140000,taker,"{'currency': 'RENDER', 'cost': 0.00304645}","{'id': 'bb919604-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.00304645}]",zonda
4,bb919605-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.11,2.000000,42.220000,taker,"{'currency': 'RENDER', 'cost': 0.004}","{'id': 'bb919605-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.004}]",zonda
5,bb919606-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.12,2.000000,42.240000,taker,"{'currency': 'RENDER', 'cost': 0.004}","{'id': 'bb919606-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.004}]",zonda
6,bb919607-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.14,69.161306,1462.070000,taker,"{'currency': 'RENDER', 'cost': 0.13832262}","{'id': 'bb919607-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.13832262}]",zonda
7,732a3275-9fa5-11ef-a0fa-0242ac110007,732a3274-9fa5-11ef-a0fa-0242ac110007,1731271842234,2024-11-10T20:50:42.234Z,ETH/PLN,limit,buy,12630.00,0.395883,4999.999890,taker,"{'currency': 'ETH', 'cost': 0.00178148}","{'id': '732a3275-9fa5-11ef-a0fa-0242ac110007',...","[{'currency': 'ETH', 'cost': 0.00178148}]",zonda
8,7678b8e9-a1cf-11ef-a0fa-0242ac110007,7678b8e8-a1cf-11ef-a0fa-0242ac110007,1731509788970,2024-11-13T14:56:28.970Z,ETH/PLN,limit,buy,13658.50,0.732147,10000.029936,taker,"{'currency': 'ETH', 'cost': 0.00329467}","{'id': '7678b8e9-a1cf-11ef-a0fa-0242ac110007',...","[{'currency': 'ETH', 'cost': 0.00329467}]",zonda
9,89fe087a-b3f9-11ef-86d5-0242ac110005,fba763f7-b3f7-11ef-86d5-0242ac110005,1733506981512,2024-12-06T17:43:01.512Z,BTC/PLN,limit,sell,401900.00,0.100000,40190.000000,maker,"{'currency': 'PLN', 'cost': 120.57}","{'id': '89fe087a-b3f9-11ef-86d5-0242ac110005',...","[{'currency': 'PLN', 'cost': 120.57}]",zonda


## Wyświetlenie trazakcji

In [16]:
## Display transactions as a table
display(all_transactions_df)

## Display transaction in print
# print(all_transactions_df)


,id,order,timestamp,datetime,symbol,type,side,price,amount,cost,takerOrMaker,fee,info,fees,exchange
0,bb916ef1-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.18,6.441520,129.989876,taker,"{'currency': 'RENDER', 'cost': 0.01288305}","{'id': 'bb916ef1-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.01288305}]",zonda
1,bb916ef2-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.19,43.960000,887.552400,taker,"{'currency': 'RENDER', 'cost': 0.08792}","{'id': 'bb916ef2-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.08792}]",zonda
2,bb916ef3-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,20.20,119.000000,2403.800000,taker,"{'currency': 'RENDER', 'cost': 0.238}","{'id': 'bb916ef3-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.238}]",zonda
3,bb919604-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.10,1.523223,32.140000,taker,"{'currency': 'RENDER', 'cost': 0.00304645}","{'id': 'bb919604-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.00304645}]",zonda
4,bb919605-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.11,2.000000,42.220000,taker,"{'currency': 'RENDER', 'cost': 0.004}","{'id': 'bb919605-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.004}]",zonda
5,bb919606-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.12,2.000000,42.240000,taker,"{'currency': 'RENDER', 'cost': 0.004}","{'id': 'bb919606-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.004}]",zonda
6,bb919607-9dd5-11ef-80a7-0242ac110004,bb916ef0-9dd5-11ef-80a7-0242ac110004,1731072677225,2024-11-08T13:31:17.225Z,RENDER/PLN,limit,buy,21.14,69.161306,1462.070000,taker,"{'currency': 'RENDER', 'cost': 0.13832262}","{'id': 'bb919607-9dd5-11ef-80a7-0242ac110004',...","[{'currency': 'RENDER', 'cost': 0.13832262}]",zonda
7,732a3275-9fa5-11ef-a0fa-0242ac110007,732a3274-9fa5-11ef-a0fa-0242ac110007,1731271842234,2024-11-10T20:50:42.234Z,ETH/PLN,limit,buy,12630.00,0.395883,4999.999890,taker,"{'currency': 'ETH', 'cost': 0.00178148}","{'id': '732a3275-9fa5-11ef-a0fa-0242ac110007',...","[{'currency': 'ETH', 'cost': 0.00178148}]",zonda
8,7678b8e9-a1cf-11ef-a0fa-0242ac110007,7678b8e8-a1cf-11ef-a0fa-0242ac110007,1731509788970,2024-11-13T14:56:28.970Z,ETH/PLN,limit,buy,13658.50,0.732147,10000.029936,taker,"{'currency': 'ETH', 'cost': 0.00329467}","{'id': '7678b8e9-a1cf-11ef-a0fa-0242ac110007',...","[{'currency': 'ETH', 'cost': 0.00329467}]",zonda
9,89fe087a-b3f9-11ef-86d5-0242ac110005,fba763f7-b3f7-11ef-86d5-0242ac110005,1733506981512,2024-12-06T17:43:01.512Z,BTC/PLN,limit,sell,401900.00,0.100000,40190.000000,maker,"{'currency': 'PLN', 'cost': 120.57}","{'id': '89fe087a-b3f9-11ef-86d5-0242ac110005',...","[{'currency': 'PLN', 'cost': 120.57}]",zonda


## Zapis transakcji do CSV

In [14]:
# Creata a directory for the CSV files in home directory ~/kryptorozliczator/csv
import os

csv_dir = os.path.expanduser("~/kryptorozliczator/csv")
os.makedirs(csv_dir, exist_ok=True)

# Create a CSV file for the transactions
csv_file = os.path.join(csv_dir, "transactions.csv")
all_transactions_df.to_csv(csv_file, index=False)

# Display the CSV file
# display(pd.read_csv(csv_file))

## Obliczenie przychodów i kosztów

In [18]:
df = all_transactions_df

# Grupowanie transakcji na kupno i sprzedaż
buys = df[df['side'] == 'buy']
sells = df[df['side'] == 'sell']

# Obliczanie sum
total_buy_cost = buys['cost'].sum()  # Suma kosztów zakupów
total_sell_revenue = sells['cost'].sum()  # Suma przychodów ze sprzedaży

# Obliczanie sumy opłat
fee_pln = sells[sells['fee'].apply(lambda x: x['currency'] == 'PLN')]['fee'].apply(lambda x: float(x['cost'])).sum()

# Tworzenie podsumowania
summary = pd.DataFrame({
    'Kategoria': ['Koszty zakupu', 'Przychody ze sprzedaży', 'Opłaty (PLN)', 'Wynik'],
    'Wartość (PLN)': [
        total_buy_cost,
        total_sell_revenue,
        fee_pln,
        total_sell_revenue - total_buy_cost - fee_pln
    ]
})

# Formatowanie liczb do 2 miejsc po przecinku
summary['Wartość (PLN)'] = summary['Wartość (PLN)'].round(2)

# Wyświetlenie podsumowania
print("\nPodsumowanie transakcji:")
print(summary)

# Podsumowanie per para walutowa
print("\nPodsumowanie per para walutowa:")
for symbol in df['symbol'].unique():
    symbol_data = df[df['symbol'] == symbol]
    buys = symbol_data[symbol_data['side'] == 'buy']
    sells = symbol_data[symbol_data['side'] == 'sell']
    
    buy_volume = buys['amount'].sum()
    sell_volume = sells['amount'].sum()
    buy_cost = buys['cost'].sum()
    sell_revenue = sells['cost'].sum()
    
    print(f"\n{symbol}:")
    print(f"Zakupiono: {buy_volume:.8f}")
    print(f"Sprzedano: {sell_volume:.8f}")
    print(f"Koszt zakupów: {buy_cost:.2f} PLN")
    print(f"Przychód ze sprzedaży: {sell_revenue:.2f} PLN")


Podsumowanie transakcji:
                Kategoria  Wartość (PLN)
0           Koszty zakupu       20000.04
1  Przychody ze sprzedaży       40190.00
2            Opłaty (PLN)         120.57
3                   Wynik       20069.39

Podsumowanie per para walutowa:

RENDER/PLN:
Zakupiono: 244.08604846
Sprzedano: 0.00000000
Koszt zakupów: 5000.01 PLN
Przychód ze sprzedaży: 0.00 PLN

ETH/PLN:
Zakupiono: 1.12802982
Sprzedano: 0.00000000
Koszt zakupów: 15000.03 PLN
Przychód ze sprzedaży: 0.00 PLN

BTC/PLN:
Zakupiono: 0.00000000
Sprzedano: 0.10000000
Koszt zakupów: 0.00 PLN
Przychód ze sprzedaży: 40190.00 PLN
